In [2]:
import subprocess
from pydub import AudioSegment


def extract_audio_from_video(video_path, audio_path):
    command = [
        "ffmpeg",
        "-i",
        video_path,
        "-vn",
        audio_path,
    ]
    subprocess.run(command)


track = AudioSegment.from_mp3("../files/audio.mp3")

ten_minutes = 10 * 60 * 1000

In [3]:
import math

chunks = math.ceil(len(track) / ten_minutes)

for i in range(chunks):
    start_time = i * ten_minutes
    end_time = (i + 1) * ten_minutes

    chunk = track[start_time:end_time]

    chunk.export(f"../files/chunks/chunk_{i}.mp3", format="mp3")


def cut_audio_in_chunks(audio_path, chunk_size, chunks_folder):
    track = AudioSegment.from_mp3(audio_path)
    chunk_len = chunk_size * 60 * 1000
    chunks = math.ceil(len(track) / chunk_len)
    for i in range(chunks):
        start_time = i * chunk_len
        end_time = (i + 1) * chunk_len
        chunk = track[start_time:end_time]
        chunk.export(
            f"./{chunks_folder}/chunk_{i}.mp3",
            format="mp3",
        )


In [4]:
track.duration_seconds

1031.685804988662

In [5]:
from typing import final
import openai
import glob


def transcribe_chunks(chunk_folder, destination):
    files = glob.glob(f"{chunk_folder}/*.mp3")
    files.sort()
    for file in files:
        with open(file, "rb") as audio_file, open(destination, "a") as text_file:
            transcript = openai.Audio.transcribe(
                "whisper-1",
                audio_file,
            )
            text_file.write(transcript["text"])

# def transcribe_chunks(chunk_folder, destination):
#     files = glob.glob(f"{chunk_folder}/*.mp3")
#     final_transcript = ""
#     for file in files:
#         with open(file, "rb") as audio_file:
#             transcript = openai.Audio.transcribe(
#                 "whisper-1",
#                 audio_file,
#             )
#             final_transcript += transcript["text"]
#     with open(destination, "w") as file:
#         file.write(final_transcript)

In [6]:
transcribe_chunks("../files/chunks", "../files/transcript.txt")